In [ ]:
import pymc3 as pm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import theano
from pymc3.distributions import Continuous
from pymc3.distributions import distribution
%matplotlib inline

In [ ]:
frequencies = np.loadtxt('/Users/Rhys/Documents/PhD/Maths_docs/PyMC3/frequencies_test.txt')
frequencies = 2*frequencies[1:]
frequencies.shape

In [ ]:
def lambda0(neta , mass):        
    return 3/(128 * neta * (np.pi*mass)**(5/3))
def phase(tc , phic , f, neta , mass ):
    rh_term = lambda0(neta , mass)*f**(-4/3)
    return 2*np.pi*f*tc + phic + rh_term
def strain(amplitude , mass , distance, neta , f ,tc , phic ):
    exponent = 1j*phase(tc, phic, f, neta ,lambda0(neta, mass)) + 1j*(np.pi/4) 
    return (amplitude * mass**(5/6))/(distance*np.pi**(2/3))* ((5*neta)/(24))**(-7/6)*np.exp(exponent)

In [ ]:
waveform = strain(10**-3, 30 , 1 , 0.25 , 2*frequencies , 0 , 0)
waveform1 = strain(10**-3, 10 , 1 , 0.25 , 2*frequencies , 0 , 0)

diff = np.linalg.norm(waveform - waveform1)
diff

In [ ]:
waveform = strain(10**-3, 30 , 1 , 0.25 , 2*frequencies , 0 , 0)
plt.plot(2*frequencies,waveform)
plt.show()

In [ ]:
from theano.compile.nanguardmode import NanGuardMode
mode = NanGuardMode(nan_is_error=True, inf_is_error=True, big_is_error=True)

In [ ]:
with pm.Model() as gw_model:
    neta = pm.Normal('neta' , mu = 0.1 , sd = 0.2)
    mass = pm.Normal('mass' , mu = 25 , sd = 100)
    amplitude 
    phase = 3/(128 * neta * (np.pi*mass)**(5/3))*frequencies**(-4/3)
    exponent = 1j*phase + 1j*(np.pi/4) 
    strain = (10**-3 * mass**(5/6))/(np.pi**(2/3))* ((5*neta)/(24))**(-7/6)*np.exp(exponent)
    diff = np.linalg.norm(waveform - strain)
    likelihood = pm.Normal('obs' , mu = diff
                           , sd =1 , observed = np.random.normal( 0 , 1, len(waveform)))

In [ ]:
right = gw_model.logp({'mass': 30 , 'neta':0.25})
wrong = gw_model.logp({'mass': 100 , 'neta':0.1})


In [ ]:
print(right)
print(wrong)

In [ ]:
with gw_model : 
    approx = pm.fit(method = 'advi',  n = 50000)
#     trace = pm.sample(2000 , method = '' , tune = 500)
    